<a href="https://colab.research.google.com/github/ericslevenson/arctic-surface-water/blob/main/S2maskedmosaic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
Created 7-22-2022

author: @ericslevenson

Produce cloud-masked Sentinel-2 mosaics in Earth Engine and export for prediction
'''

In [ ]:
'''
Instructions:
Enter the ROI and time period and produce either: (a) a BGRN cloud-masked mosaic
from the clearest day in the time period, or (b) a BGRN median composite of the 
images above a certain clearness threshold. Method A is preferable. Export to
Google Drive for UNET Lake prediction. 
'''

In [ ]:
'''TODO
--Keep manual inspection, but add image to imagecollection then batch export
'''

In [ ]:
# Authenticate private account (only required for exporting to drive/gee/gcp)
from google.colab import auth 
auth.authenticate_user()

# Earth Engine setup
import ee # Trigger the authentication flow.
ee.Authenticate()
ee.Initialize() # Initialize the library.

# Google Drive setup (if needed)
from google.colab import drive
drive.mount('/content/drive')

# Some common imports
from IPython.display import Image
import folium

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=QTcZ7OuklsjaHlpgCYcLuj2-HNgfCIYoQrqNTd5WqlI&tc=OYsnwkWHNShVLdip5fSyT9c77Qr3U-9PwiOFCQjPCkE&cc=cM0hU6Dzm3_xkshWeSAmjSf_-dg325e-QutzdqdZl3U

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJqF9GqzuokfeJnHKOCt_lLnqQsOkWsXmKjHeBnI6e_w59ACKAmId8A

Successfully saved authorization token.
Mounted at /content/drive


In [ ]:
### Inputs
roi = ee.FeatureCollection('projects/ee-eric-levenson/assets/woodwellS2/yukon_flats_AOI')
cell = '5N_CCC'
#roi = grid.filter('ID_1 == "5N_CCC"')
#roi = roi.reproject(epsg)
polygon = ee.Geometry.Polygon(roi.geometry().bounds().getInfo()['coordinates'][0]) 
# Set time period dates 
start = ee.Date('2019-05-01')
finish = ee.Date('2021-10-30')
coverage = .8 # ratio of roi covered in clear pixels
epsg = 'EPSG:32605'
projection = ee.Projection('EPSG:32606').atScale(10)

In [ ]:
polygon.area().getInfo()

6685079656.686183

In [ ]:
## ***IMAGE PRE-PROCESSING METHODS***

# Mask clouds in Sentinel-2
def maskS2clouds(image):
  '''Takes an input and adds two bands: cloud mask and clear mask'''
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  clear_mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)).rename('clear_mask')
  cloud_mask = qa.bitwiseAnd(cloudBitMask).eq(1).And(qa.bitwiseAnd(cirrusBitMask).eq(1)).rename('cloud_mask')
  return image.addBands([cloud_mask,clear_mask])

# Apply cloud mask to other bands
def applyMask(image):
  img = image.updateMask(image.select('clear_mask'))
  return img

# Clip image
def clip_image(image):
  '''Clips to the roi defined at the beginning of the script'''
  return image.clip(roi)

  # Mosaic images by date, orbit, - basically combines images together that were taken on the same day 
def mosaicBy(imcol):
  '''Takes an image collection (imcol) and creates a mosaic for each day
  Returns: An image collection of daily mosaics'''
  #return the collection as a list of images (not an image collection)
  imlist = imcol.toList(imcol.size())
  # Get all the dates as list
  def imdate(im):
    date = ee.Image(im).date().format("YYYY-MM-dd")
    return date
  all_dates = imlist.map(imdate)
  # get all orbits as list
  def orbitId(im):
    orb = ee.Image(im).get('SENSING_ORBIT_NUMBER')
    return orb
  all_orbits = imlist.map(orbitId)
  # get all spacecraft names as list
  def spacecraft(im):
    return ee.Image(im).get('SPACECRAFT_NAME')
  all_spNames = imlist.map(spacecraft)
  # this puts dates, orbits and names into a nested list
  concat_all = all_dates.zip(all_orbits).zip(all_spNames);
  # here we unnest the list with flatten, and then concatenate the list elements with " "
  def concat(el):
    return ee.List(el).flatten().join(" ")
  concat_all = concat_all.map(concat)
  # here, just get distinct combintations of date, orbit and name
  concat_unique = concat_all.distinct()
  # mosaic
  def mosaicIms(d):
    d1 = ee.String(d).split(" ")
    date1 = ee.Date(d1.get(0))
    orbit = ee.Number.parse(d1.get(1)).toInt()
    spName = ee.String(d1.get(2))
    im = imcol.filterDate(date1, date1.advance(1, "day")).filterMetadata('SPACECRAFT_NAME', 'equals', spName).filterMetadata('SENSING_ORBIT_NUMBER','equals', orbit).mosaic()
    return im.set(
        "system:time_start", date1.millis(),
        "system:date", date1.format("YYYY-MM-dd"),
        "system:id", d1)
  mosaic_imlist = concat_unique.map(mosaicIms)
  return ee.ImageCollection(mosaic_imlist)

def reprojectMosaic(image):
  '''Reproject to UTM. A future function should take the image location and return
  the UTM zone. For now, I'm manually entering the EPSG code.'''
  image_projected = image.reproject(epsg).atScale(10)
  return image_projected

# Get percentile cover   
def getCover(image):
  '''calculates percentage of the roi covered by the clear mask. NOTE: this function
  calls the global totPixels variable that needs to be calculated in the main script.'''
  actPixels = ee.Number(image.updateMask(image.select('clear_mask')).reduceRegion(
      reducer = ee.Reducer.count(),
      scale = 100,
      geometry = roi,
      maxPixels=1e12,
      ).values().get(0))
  # calculate the perc of cover OF CLEAR PIXELS 
  percCover = actPixels.divide(totPixels).multiply(100).round()
  # number as output
  return image.set('percCover', percCover,'actPixels',actPixels)

# Select bgrn bands
def selectBands(image):
  img = image.select(['B2', 'B3', 'B4', 'B8'])
  return img

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [ ]:
##Based on S2 Tiles::

In [ ]:
# Original method...best image per month
start = '2020-08-01'
finish = '2020-09-15'
eestart = ee.Date(start)
eefinish = ee.Date(finish)
coverage = 75 # ratio of roi covered in clear pixels
## Image Pre-Processing KEEP THIS ORIGINAL
images = ee.ImageCollection('COPERNICUS/S2').filterBounds(roi).filterDate(start,finish).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',50)) # Get Images
images_all = mosaicBy(images) # Mosaic images
images_all = images_all.map(maskS2clouds) # Create cloud/clear masks
images_all = images_all.map(clip_image) # Clip to roi
images_all = images_all.map(applyMask) # mask other bands that are cloudy
image_mask = images.select('B2').mean().clip(roi).gte(0) #First, calculate total number of pixels
totPixels = ee.Number(image_mask.reduceRegion(
    reducer = ee.Reducer.count(),
    scale = 100,
    geometry = roi,
    maxPixels = 1e12
    ).values().get(0))
images_all = images_all.map(getCover) # Add percent cover as an image property
images_all = images_all.filterMetadata('percCover','greater_than',50) # remove images covering less than 50% of the ROI)
images_all = images_all.map(selectBands) # reduce to bgrn bands
percCovers = images_all.aggregate_array('percCover').getInfo() # get non EE list of percent covers
maxim = max(percCovers) # get maximum percent cover
best_image = images_all.filterMetadata('percCover','equals',maxim) # get image with max percent cover
img = ee.Image(best_image.first())
#img = reprojectMosaic(img)
date = img.getInfo()['id'][0] # get date for export
date = date.replace('-','')
filename = str(date+'YKF_BGRN_SR')

In [ ]:
percCovers

[98, 56, 80, 72, 85, 90, 100, 69, 89, 77, 100, 94, 85]

In [ ]:
filename

'20200817YKF_BGRN_SR'

In [ ]:
## Visualize to double check ##

# Define the visualization parameters.
image_viz_params = {
    'bands': ['B8', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': [0.95, 1.1, 1]
}
# Define a map centered on San Francisco Bay.
map_l8 = folium.Map(zoom_start=10)
# Add the image layer to the map and display it.
map_l8.add_ee_layer(img, image_viz_params, 'true color composite')
folium.GeoJson(roi.geometry().getInfo()).add_to(map_l8)
display(map_l8)

In [ ]:
totPixels.getInfo()

1877334

In [ ]:
img.getInfo()

In [ ]:
# Exporting asset to drive example (region could be specified)
task = ee.batch.Export.image.toDrive(**{
    'image': img,
    'description': filename,
    'folder':'S2mosaics',
    'fileFormat': 'GeoTIFF',
    'scale': 10,
    'region': polygon,
    'maxPixels': 1e12
})
task.start()
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id: 7T5PEUOEJ3E3IZJ6CYGFJAB6).
Polling for task (id

In [ ]:
# Second method...composite of best images per month
start = '2020-07-01'
finish = '2020-07-31'
eestart = ee.Date(start)
eefinish = ee.Date(finish)
coverage = 75 # ratio of roi covered in clear pixels
## Image Pre-Processing KEEP THIS ORIGINAL
images = ee.ImageCollection('COPERNICUS/S2').filterBounds(roi).filterDate(start,finish).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',50)) # Get Images
images_all = mosaicBy(images) # Mosaic images
images_all = images_all.map(maskS2clouds) # Create cloud/clear masks
images_all = images_all.map(clip_image) # Clip to roi
images_all = images_all.map(applyMask) # mask other bands that are cloudy
image_mask = images.select('B2').mean().clip(roi).gte(0) #First, calculate total number of pixels
totPixels = ee.Number(image_mask.reduceRegion(
    reducer = ee.Reducer.count(),
    scale = 100,
    geometry = roi,
    maxPixels = 1e12
    ).values().get(0))
images_all = images_all.map(getCover) # Add percent cover as an image property
images_all = images_all.filterMetadata('percCover','greater_than', coverage) # remove images covering less than 50% of the ROI)
images_all = images_all.map(selectBands) # reduce to bgrn bands
#percCovers = images_all.aggregate_array('percCover').getInfo() # get non EE list of percent covers
#bestCovers = percCovers[0:3] # Top 3 percentages
#best_images = images_all.filterMetadata('percCover','equals',bestCovers) # THIS ISNT WORKING
img = images_all.reduce(ee.Reducer.median()) # Median composite of best 3 images
date = start[0:7]
date = date.replace('-','')
filename = str(date+'_ykd_BGRN_SR')

In [ ]:
## Visualize to double check ##

# Define the visualization parameters.
image_viz_params = {
    'bands': ['B8_median', 'B3_median', 'B2_median'],
    'min': 0,
    'max': 3000,
    'gamma': [0.95, 1.1, 1]
}
# Define a map centered on San Francisco Bay.
map_l8 = folium.Map(zoom_start=10)
# Add the image layer to the map and display it.
map_l8.add_ee_layer(img, image_viz_params, 'true color composite')
folium.GeoJson(roi.geometry().getInfo()).add_to(map_l8)
display(map_l8)

In [ ]:
# ************Assemble a feature collection of images I want and then loop through the export************